<a href="https://colab.research.google.com/github/Jenna-Williams/Python-Class/blob/master/China_NavInfo_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#China NavInfo and B-F Discount Data Analysis (F20 - F21)


---


##Overview


---


Currently, B-F data is limited regarding where products are being sold at the retail-level in China markets. To help bridge the gap between distributors and consumers, location data was purchased from a third party, NavInfo. 

This colab notebook utilizes multiple datasets to get a more accurate look at sales by channels, including:
* NavInfo location data (i.e.  Province, City, Account Name, Channel)
* B-F Discount Data for F20 and F21
* Discount Reconciliation Data for F20 and F21

Ultimately, we are interested in what additional information we can gain when combining the NavInfo data with our discount data. Key fields we use from our discount data include volume, discounts, and brand size.
From the NavInfo dataset we focus on location data such as city and outlets ("Account Name") which can be linked to the customers.

The key question we seek to answer in this analysis, is "how do customer sales relate to entity and outlet locations across China?"


---


##Methods


---


### Translation (Chinese to English)
Key information is translated (from Chinese to English) for the NavInfo and B-F datasets using the Google Translate API. This is done using the Python module 'googletrans'. 
### Matching
After the translation, this data is added back to the original dataframe. Then we combine all datasets into a new dataframe for further analysis.

### Additional Info:
* Key information from BF and NavInfo files is translated: Chinese → English via googletrans
* Key fields translated / used (“NavInfo Data” tab of "NavInfo Data.xlsx" file):  
    1.   Account Name (Customer Name)
    2.   City
    3.   Province
    4.   District
* Add translated columns back to original dataframe.
* Channel and customer mapping:
 * Select [Account Name] from B-F Discount Data (F20-F21) df.
 * Match to [Account Name] in NavInfo df.
 * Compare Customber by Brand Size.
* Create results tables (similar to pivot tables from "NavInfo Data.xlsx" file to show:
 * Overview of NavInfo Outlets (by Province and City]
 * Overview of Customer Distribution to Outlets.
 * Top/Bottom 10 Accounts (By Discounts)
 * Example: In Beijing, the top 12 customers accounted for over 50 percent of discounts.
 * Brand Pack Size Top/Bottom Accounts
 * Example: Brannd Size - nearly 80% of discounts were for JD 700ml in Beijing.
* Brand Pack Size by Customer
* Listing and Selling Status by Customer (E.g. what listing opportunities would there be with existing customers?)
* In the future, what could we expect in specific sub-channels for listings? For example, might we expect 'core listing in KTVs' to perform going forward?
---


##Results


---






In [1]:
# Import Packages
!pip install -q xlrd
import xlrd
import io
import json

# Import necessary packages for Data Analytics/Visualization
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import rpy2 # to use R in python env
import datetime as dt

# Install in-line ggplot package for visualization
!pip install tslib
!pip install ggplot
#from ggplot import *

# Import Google Translator API package
!pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator, constants
from pprint import pprint
#!pip install git+https://github.com/BoseCorp/py-googletrans.git --upgrade
#!pip install git+https://github.com/lushan88a/google_trans_new.git --upgrade
#!pip install google_trans_new
#from google_trans_new import google_translator

import six
from google.cloud import translate_v2 as translate

!pip install translate
import translate

# Install / Import OCR libraries
!pip install pytesseract Pillow
!pip install opencv-python
!pip install pdf2image
!pip install PyPDF2

import cv2
from pdf2image import convert_from_path
import PyPDF2
import os
import sys

     |████████████████████████████████| 2.2 MB 3.8 MB/s 
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 5.5 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=de79ef16b5626d743cc796792beb62c600a3116acd1cb5f97b4d226cd51c2168
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=7147adc95acb934a8a7c12a811fc810e21f9b379674d211ce66cd0de757dfec5
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract
     |████████████████████████████████| 77 kB 2.9 MB/s 
  Cr

##Additional Resources

Google Translation API:
* Official Google API Documentation: https://cloud.google.com/translate/docs
* Python googletrans Documentation: https://github.com/ssut/py-googletrans


Articles:
* Translate Pandas df: https://towardsdatascience.com/translate-a-pandas-data-frame-using-googletrans-library-fb0aa7fca592
* Googletrans: http://zetcode.com/python/googletrans/
* Using Google Translator in Python: https://www.codeproject.com/Tips/1236705/How-to-Use-Google-Translator-in-Python
* Translate Column: https://stackoverflow.com/questions/58350457/excel-sheet-translation-using-python
* Pandas: https://medium.com/analytics-vidhya/translate-list-and-pandas-data-frame-using-googletrans-library-in-python-f28b8cb84f21
* Read the docs: https://py-googletrans.readthedocs.io/en/latest/


In [24]:
# Check python version
!python --version

Python 3.7.12


Mount Google Drive 

In [3]:
# Mounting Google Drive locally
# Mount MyDrive on runtime using authoriazation code.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/China NavInfo Data"

'13 cities account list 20210331.xlsx'
'BF Channel Definition - 20191213.gsheet'
'Brown-Forman Scotch Products Distribution Agreement 百富门单一麦芽威士忌经销协议 [EXECUTION VERSION] Fully Signed NOA.pdf'
'Discount Reconciliation Report for F21 Apr 20210515.xlsx'
'Discounts F20.xlsx'
'Discounts F21.xlsx'
'NavInfo and F20 Discount Data.xlsx'
 NavInfo_Data_Translated.xlsx
 output


In [5]:
# Import NavInfo Raw data sheet from "NavInfo and F20 Discount Data.xlsx" Excel file
df_nav = pd.read_excel('/content/drive/My Drive/China NavInfo Data/NavInfo and F20 Discount Data.xlsx', sheet_name = 'NavInfo Raw data')
#df_nav.head()

In [6]:
# Describe Data
#df_nav.describe()

#Google Translator API

Get Supported Languages
Translate Columns
Translate Rows
Join Multiple Datasets

1.   Get Supported Languages
2.   Detect Language
3.   Translate Columns/Rows
4.   Join Multiple Datasets



In [7]:
"""
# Using Google translator API (googletrans package)
# List supported lanugages

from googletrans import LANGUAGES

for lang in LANGUAGES:
  print(f'{lang} - {LANGUAGES[lang]}')
"""

"\n# Using Google translator API (googletrans package)\n# List supported lanugages\n\nfrom googletrans import LANGUAGES\n\nfor lang in LANGUAGES:\n  print(f'{lang} - {LANGUAGES[lang]}')\n"

In [8]:
# Get supported languages and corresponding codes

#pd.set_option('max_colwidth', 300)
pd.set_option("display.max_rows", None, "display.max_columns", None)

lang_df = pd.DataFrame.from_dict(googletrans.LANGUAGES, orient='index', columns = ['Language'])
lang_df

,Language
af,afrikaans
sq,albanian
am,amharic
ar,arabic
hy,armenian
az,azerbaijani
eu,basque
be,belarusian
bn,bengali
bs,bosnian


In [9]:
# Listing supported languages
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [10]:
# Find code for english and chinese
lang_df[lang_df.Language.isin([
                               'english', 
                               'chinese (simplified)', 
                               'chinese (traditional)'
                               ])]

,Language
zh-cn,chinese (simplified)
zh-tw,chinese (traditional)
en,english


##Testing Translations

In [11]:
# testing 
translator = Translator()  
translate_text = translator.translate('สวัสดีจีน',lang_tgt='en')  
print(translate_text)

Translated(src=th, dest=en, text=hello china, pronunciation=None, extra_data="{'translat...")


In [12]:
# Test the program to detect language

from googletrans import Translator

detector = Translator()
dec_lan = detector.detect('이 문장은 한글로 쓰여졌습니다.')

print(dec_lan)

Detected(lang=ko, confidence=1)


In [13]:
# Test - get source and destination langauge, and print
trans = Translator()
t = trans.translate(
    '이 문장은 한글로 쓰여졌습니다.'
)

# See source language
print(f'Source: {t.src}')
# See destination lanugage
print(f'Destination: {t.dest}')
# See translated text
print(f'{t.origin} -> {t.text}')

Source: ko
Destination: en
이 문장은 한글로 쓰여졌습니다. -> This sentence is written in Korean.


In [14]:
trans = Translator()
#t = trans.translate(('縣',	'城市',	'省',	'用戶名', '地址',	'分类1',	'分类2',	
#                     '分类3',	'分类编码',	'业态'), 
#                     src ='zh-CN', dest='en')
t = trans.translate('縣	城市	省	用戶名 地址	分类1	分类2	分类3	分类编码	业态', src ='zh-CN', dest='en')
print(f'Source: {t.src}')
print(f'Destination: {t.dest}')
print(f'{t.origin} -> {t.text}')

Source: zh-CN
Destination: en
縣	城市	省	用戶名 地址	分类1	分类2	分类3	分类编码	业态 -> County City Province User Name Address Category 1 Category 2 Category 3 Category Code Business Type


In [15]:
# Test possible mistakes
trans = Translator()
t = trans.translate(
    '이 문장은 한글로 쓰여졌습니다.', src='ko', dest ='en'
)

# See source language
print(f'Source: {t.src}')
# See destination lanugage
print(f'Destination: {t.dest}')
# See translated text
print(f'{t.origin} -> {t.text}')

# Check translation - dictionary for possible mistakes and possible translations
pm = t.extra_data['possible-mistakes']
pt = t.extra_data['possible-translations']
print(f'Possible Mistakes: {pm}')
print(f'Possible Translations: {pt}')

Source: ko
Destination: en
이 문장은 한글로 쓰여졌습니다. -> This sentence is written in Korean.
Possible Mistakes: None
Possible Translations: [['이 문장은 한글로 쓰여졌습니다.', None, [['This sentence is written in Korean.', 0, True, False, [3]], ['This sentence is written in Hangul.', 0, True, False, [0]]], [[0, 17]], '이 문장은 한글로 쓰여졌습니다.', 0, 0]]


In [16]:
# Test - get source and destination lanugage, and print
trans = Translator()
t = trans.translate('用戶名')

# See source language
print(f'Source: {t.src}')
# See destination lanugage
print(f'Destination: {t.dest}')
# See translated text
print(f'{t.origin} -> {t.text}')

Source: zh-CN
Destination: en
用戶名 -> username


In [17]:
# Test possible mistakes
# Test - get source and destination lanugage, and print
trans = Translator()
t = trans.translate(('分类1, 用戶 名'), src ='zh-CN', dest = 'en')

# See source language
print(f'Source: {t.src}')
# See destination lanugage
print(f'Destination: {t.dest}')
# See translated text
print(f'{t.origin} -> {t.text}')

# Check translation - dictionary for possible mistakes and possible translations
pm = t.extra_data['possible-mistakes']
pt = t.extra_data['possible-translations']
print(f'Possible Mistakes: {pm}')
print(f'Possible Translations: {pt}')

Source: zh-CN
Destination: en
分类1, 用戶 名 -> Category 1, Username
Possible Mistakes: None
Possible Translations: [['分类1, 用戶 名', None, [['Category 1, Username', 0, True, False, [3]], ['Category 1, the user name', 0, True, False, [0]]], [[0, 9]], '分类1, 用戶 名', 0, 0]]


In [18]:
"""
# Translates text into the target language.
# NOTE: Target must be an ISO 639-1 language code. See https://g.co/cloud/translate/v2/translate-reference#supported_languages

def translate_text(target, text):
  translate_client = translate.Client()
  
  if isinstance(text, six.binary_type):
    text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
"""

'\n# Translates text into the target language.\n# NOTE: Target must be an ISO 639-1 language code. See https://g.co/cloud/translate/v2/translate-reference#supported_languages\n\ndef translate_text(target, text):\n  translate_client = translate.Client()\n  \n  if isinstance(text, six.binary_type):\n    text = text.decode("utf-8")\n\n    # Text can also be a sequence of strings, in which case this method\n    # will return a sequence of results for each text.\n    result = translate_client.translate(text, target_language=target)\n\n    print(u"Text: {}".format(result["input"]))\n    print(u"Translation: {}".format(result["translatedText"]))\n    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))\n'

In [19]:
"""
# Testing googletrans API
# Translate word doc file and save result as doc (keeping same file format).

#:param filename: word doc file
#:param destination='zh-CN':
#:param mix=True: if True, will have original language and target language into the same doc. paragraphs by paragraphs.

def translate_doc(filename, destination='zh-CN', mix=True):
  def tx(t): return Translator().translate(t, dest=destination).text
  doc = Document(filename)
  for p in doc.paragraphs:
    txd = tx(p.text)
    
    p.text = p.text + ('\n' + txd if mix else '')

    for table in doc.tables:
      for row in table.rows:
        for cell in row.cells:
          txd = tx(cell.text)
          p.text = cell.text + ('\n' + txd if mix else '')

    f = filename.replace('.doc', destination.lower() + '.doc')
    doc.save(f) 
"""

"\n# Testing googletrans API\n# Translate word doc file and save result as doc (keeping same file format).\n\n#:param filename: word doc file\n#:param destination='zh-CN':\n#:param mix=True: if True, will have original language and target language into the same doc. paragraphs by paragraphs.\n\ndef translate_doc(filename, destination='zh-CN', mix=True):\n  def tx(t): return Translator().translate(t, dest=destination).text\n  doc = Document(filename)\n  for p in doc.paragraphs:\n    txd = tx(p.text)\n    \n    p.text = p.text + ('\n' + txd if mix else '')\n\n    for table in doc.tables:\n      for row in table.rows:\n        for cell in row.cells:\n          txd = tx(cell.text)\n          p.text = cell.text + ('\n' + txd if mix else '')\n\n    f = filename.replace('.doc', destination.lower() + '.doc')\n    doc.save(f) \n"

In [20]:
# Testing googletrans on one column 'Tier 1 WS: Account Name' in 'testing' sheet of "Discounts F20.xlsx"
df = pd.read_excel('/content/drive/MyDrive/China NavInfo Data/Discounts F20.xlsx', sheet_name="testing")
df.tail(10)

,Tier 1 WS: Account Name
557,中山市汇广商贸有限公司
558,中山市汇广商贸有限公司
559,中山市汇广商贸有限公司
560,中山市汇广商贸有限公司
561,中山市汇广商贸有限公司
562,中山市汇广商贸有限公司
563,中山市汇广商贸有限公司
564,中山市汇广商贸有限公司
565,中山市汇广商贸有限公司
566,中山市汇广商贸有限公司


In [21]:
# Building translator function and translating test df
translator = Translator()
translations = {}
for column in df.columns:

  # Unique elements of column

  unique_elements = df[column].unique()
  for element in unique_elements:

    # Adding all translations to a dictionary (translations)

    translations[element] = translator.translate(element).text

translations

{'中山市汇广商贸有限公司': 'Zhongshan Huiguang Trading Co., Ltd.'}

In [22]:
# Replacing translated words to orginal data frame
df.replace(translations, inplace = True)
df.head()

,Tier 1 WS: Account Name
0,"Zhongshan Huiguang Trading Co., Ltd."
1,"Zhongshan Huiguang Trading Co., Ltd."
2,"Zhongshan Huiguang Trading Co., Ltd."
3,"Zhongshan Huiguang Trading Co., Ltd."
4,"Zhongshan Huiguang Trading Co., Ltd."


# NavInfo Data

In [23]:
df_navinfo = pd.read_excel('/content/drive/MyDrive/China NavInfo Data/NavInfo Data_CN.xlsx', sheet_name="NavInfo Data_cn")
df_navinfo.head()

FileNotFoundError: ignored

In [ ]:
translator = Translator()
df_navinfo = pd.read_excel('/content/drive/MyDrive/China NavInfo Data/NavInfo Data_CN.xlsx', sheet_name="NavInfo Data_cn")
#df_navinfo['County', 'City', 'Province', 'Account Name', 'Address', 'Category 1', 'Category 2', 'Category 3', 'Channel'] = df_navinfo['縣',	'城市',	'省',	'用戶名',	'地址',	'分类1',	'分类2', '分类3',	'业态'].apply(translator.translate, src='zh-cn', dest='en').apply(getattr, args=('text',))
df_navinfo['County'] = df_navinfo['縣'].apply(translator.translate, src='zh-cn', dest='en').apply(getattr, args=('text',))
df_navinfo.head()    

### NavInfo Data Prep

In [ ]:
"""
# Read in full NavInfo data
df_navinfo_full = pd.read_excel('/content/drive/MyDrive/China NavInfo Data/NavInfo Data.xlsx', sheet_name="NavInfo Data")
df_navinfo_full.head()
# Drop navinfo df columns (in English)
df_navinfo_full.drop(['poi_id', 'Longitude', 'Latitude', 'County Code', 'City Code', 
               'Province Code', 'Brand', 'City EN',	'County EN', 'Province EN', 
               'Account Name EN', 'Category1 EN',	'Category2 EN', 'Category3 EN',	
               'On / Off-Trade', 'Channel',	'Sub-Channel', 'Banner'], axis=1, inplace=True)
df_navinfo_full.head()
"""

In [ ]:
# Articles for translating pandas df using googletrans:
# https://towardsdatascience.com/translate-a-pandas-data-frame-using-googletrans-library-fb0aa7fca592
# https://pypi.org/project/googletrans/
# http://zetcode.com/python/googletrans/
# https://www.codeproject.com/Tips/1236705/How-to-Use-Google-Translator-in-Python

# Read in NavInfo data in Chinese (using the "NavInfo Data_cn" sheet from "NavInfo Data_CN.xlsx" file)
df_navinfo_cn = pd.read_excel('/content/drive/MyDrive/China NavInfo Data/NavInfo Data_CN.xlsx', sheet_name='NavInfo Data_cn')
df_navinfo_cn.head()

In [ ]:
# Make a copy of the dataframe
df_navinfo_en = df_navinfo_cn.copy()

# Translate column names using rename function
df_navinfo_en.rename(columns=lambda x: translator.translate(x).text, inplace=True)

# Translated column names
df_navinfo_en.columns

#df_navinfo['County', 'City', 'Province', 'User Name', 'Address', 'Category 1', 'Category 2', 'Category 3', 'Category Code', 'Business Type'] = df_navinfo['縣',	'城市',	'省',	'用戶名', '地址',	'分类1',	'分类2',	'分类3',	'分类编码',	'业态].apply(translator.translate, src='zh-cn', dest='en').apply(getattr, args=('text',))

In [ ]:
# Translate rows
#translator = Translator()
translations = {}
for column in df_navinfo_en.columns:
  # unique elements of column
  unique_elements = df_navinfo_en[column].unique()
  for element in unique_elements:
    # add translation to dictionary
    translations[element] = translator.translate(element).text

#print(translations)

In [ ]:
"""
# Check translations and formatting, then modify column names
translations['city'] = 'City'
translations['username'] = 'Account Name'
translations['address'] = 'Address'
translations['Format'] = 'Channel'

print(translations)
"""

In [ ]:
# Replace in the dataframe
df_navinfo_en.replace(translations, inplace=True)
# Check modifications
df_navinfo_en.head()

### Additional Steps:
Key information from BF and NavInfo files is translated:
Chinese → English via DeepL (to overcome XLS dyslexia)
Key fields translated / used:
Account Name (Customer Name) -- from “NavInfo Raw data” tab  
City
Province
District

1.   Repeat same steps to translate  B-F Discount Data (F20 and F21) from Chinese to English.
2.   Add translated columns back to original dataframes.
3.   Combine Discount Data (F20 and F21) into new dataframe.
4.   Match Account Name from Discount Data to NavInfo Data. 



You will find the translated columns added on the right side of each raw data table. 
(--> the full data is then pulled into a Pivot each for easier manipulation)
Rough (sub-) channel and customer mapping done to align both perspectives as much as possible 
(see hidden tabs with ´mapping´ in them).
Manual: Select [customer EN name] from BF discount data and type into [Account name EN] in NavInfo Pivot to find match or proxy matches.
Manual: Copy / Paste 3. onto Customer tab
Manual: Copy/ Paste Customer by Brand Size info from BF Discount Pivot on same (4.) tab.
I was lucky enough to have Customer examples where the name of the Customer mostly matched the banner name seen be the consumer and shopper as well. 
Where that is not the case (like Noah´s I believe), some more know how would be required to perform 3-5. If we had a mapping table of that, then 3-5 could be done a bit less manual, too I believe.

Additional Info:
Some simple overview tabs are added in blue
Overview of NavInfo OUTLETS by province and city procured.
As Customer <> Outlets, it becomes obvious that we need to figure out the Outlets in a customer to put it into perspective of the total.

Customer overview
Some ´fun facts´ shown - like that the top 12 Customers account for 51% of the investments into discounts.
 
Brand Pack Size overview
More fun facts - like that 78% of the discounts relate to JD 700ml.

Brand Pack Size by Customer
How is the listing and selling status by Customer? E.g. what listing opportunities would there be with existing customers etc.
(there is a hidden tab that shows the channels the customers belong to)
Naturally, if applying some Picture of Success for what we would expect in specific sub-channels (for instance ´core listing in KTV, convenience etc. ´) could immediately move from today (here: F20) to tomorrow.



###NavInfo Data Translation

In [ ]:
# NavInfo Data - Translation
# Make copy of NavInfo dataframe
df_navinfo_en = df_navinfo.copy()

# Translate column names using rename function
df_navinfo_en.rename(columns=lambda x: translator.translate(x).text, inplace = True)

# Translated column names
df_navinfo_en.columns

In [ ]:
translations = {}
for column in df_navinfo_en.columns:

  # Unique elements of the columnn
  unique_elements = df_navinfo_en[column].unique()
  for element in unique_elements:

    # Adding all translations to dictionary (translations)
    translations[element] = translator.translate(element).text

print(translations)

In [ ]:
# Replacing translated words from translations dictionary to original dataframe
df_navinfo_en.replace(translations, inplace = True)
df_navinfo_en.head()

###NavInfo Translated Data Export

In [ ]:
# Export df to excel
df_navinfo_en.to_excel('/content/drive/MyDrive/China NavInfo Data/output/navinfo-en.xlsx', sheet_name="navinfo-en")

#Brown-Forman F20 Discount Data

In [ ]:
# Import F-F Discount Data for F20 from 'Discounts F20.xlsx' Excel file from MyDrive
df_bf_f20_discount = pd.read_excel('/content/drive/My Drive/China NavInfo Data/Discounts F20.xlsx', sheet_name = 'Data_F20')
df_bf_f20_discount.head()

### F20 Discount Data Prep

In [ ]:
# F20 Discount Data - Translation
# Drop f20 columns (in English)
df_bf_f20_discount.drop(['Account: Region',	'Sales Org',	'Region',	'Approval Status', 'Approval Status2',	
                         'Actual Date',	'Plan Date',	'Depletion month', 'Fiscal Year', 'Year+Month',	
                         'Channel',	'Group',	'Subgroup', 'Province', 'City',	'Brand',	'Brand(PT)',	
                         'Brand (Accounting)', 'Brand Code',	'Product Name: Product Name',	'Act Qty',	
                         'Actual Discount','Act Payment',	'Account: Account Owner: Alias', 'Promotion Start Date',	
                         'Promotion End Date',	'Remarks',	'T1',	'Monthly Bottle Qty', 'On Invoice Discount',	
                         'On Invoice Discount Amount', 'Payment','BF Actual Payment',	'Multiplier (9Lcs)',	'Act Qty (9Lcs)',	
                         'Multiplier (Btl)',	'Act Qty (Btl 700ml)',	'Comments	', 'Remark'], axis=1, inplace=True)
df_bf_f20_discount.head()

###F20 Data Translation

In [ ]:
# Make copy of F20 dataframe
df_bf_f20_discount_en = df_bf_f20_discount.copy()

# Translate column names using rename function
df_bf_f20_discount_en.rename(columns=lambda x: translator.translate(x).text, inplace = True)

# Translated column names
df_bf_f20_discount_en.columns

In [ ]:
# Translate to English, and add columns back into the DataFrame
# f20 dictionary
translations = {}
for column in df_bf_f20_discount_en.columns:
  # Unique elements of column
  unique_elements = df_bf_f20_discount_en[column].unique()
  for element in unique_elements:
    # Add translation to dictionary
    translations[element] = translator.translate(element).text

print(translations)

In [ ]:
# Replacing translated words from translations dictionary to original dataframe
df_bf_f20_discount_en.replace.replace(translations, inplace = True)
df_bf_f20_discount_en.head()

###F20 Translated Data Export

In [ ]:
# Export translated f20 df to excel
df_bf_f20_discount_en.to_excel('/content/drive/MyDrive/China NavInfo Data/output/f20-en.xlsx', sheet_name="f20-en")

#Brown-Forman F21 Discount Data

In [ ]:
# Import B-F Discount Data for F21 from 'Discounts F21.xlsx' Excel file from MyDrive
df_bf_f21_discount = pd.read_excel('/content/drive/My Drive/China NavInfo Data/Discounts F21.xlsx', sheet_name = 'Data_F21')
df_bf_f21_discount.head()

###F21 Discount Data Prep

In [ ]:
# Drop columns f21 df (in English)
df_bf_f21_discount.drop(['Account: Region',	'Depletion month',	'Fiscal Year',	'Year+Month', 'Channel',
                         'Group', 'Subgroup', 'Province', 'City',	'Brand',	'Brand(PT)', 'Brand (Accounting)', 
                         'Brand Code', 'JDTW', 'Brand Code', 'Product Name: Product Name',	'Act Qty',	
                         'Actual Discount',	'Act Payment',	'Account: Account Owner: Alias',	'Promotion Start Date',
                         'Promotion End Date', 'Remarks',	'T1',	'Monthly Bottle Qty', 'On Invoice Discount',	
                         'On Invoice Discount Amount', 'Payment','BF Actual Payment',	'Multiplier (9Lcs)',	
                         'Act Qty (9Lcs)', 'Multiplier (Btl)',	'Act Qty (Btl 700ml)',	'Comments	', 'Remark'], axis=1, inplace=True)
df_bf_f21_discount.head()

###F21 Data Translation

In [ ]:
# F21 Discount Data - Translation
# Make copy of F21 dataframe
df_bf_f21_discount_en = df_bf_f20_discount.copy()

# Translate column names using rename function
df_bf_f21_discount_en.rename(columns=lambda x: translator.translate(x).text, inplace = True)

# Translated column names
df_bf_f21_discount_en.columns

In [ ]:
# f21 dictionary
translations = {}
for column in df_bf_f21_discount_en.columns:
  # Unique elements of column
  unique_elements = df_bf_f21_discount_en[column].unique()
  for element in unique_elements:
    # Add translation to dictionary
    translations[element] = translator.translate(element).text

print(translations)

In [ ]:
# Modify dataframe by using replace funciton, with dictionary as input
# Modify all terms of df_bf_f21_discount
df_bf_f21_discount_en.replace(translations, inplace=True)

# Check translations
df_bf_f21_discount_en.head()

###F21 Translated Data Export

In [ ]:
# Export df to excel
df_bf_f21_discount_en.to_excel('content/drive/MyDrive/China NavInfo Data/output/f21-en.xlsx', sheet_name='f21-en')

In [ ]:
# Using Google Translate API
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print(u"Text: {}".format(result["input"]))
    print(u"Translation: {}".format(result["translatedText"]))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))


In [ ]:

text1 = '''
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
京东商城
英伦纯K量贩KTV
天猫超市-朗家园供货
天猫超市-朗家园供货
天猫超市-朗家园供货
天猫超市-朗家园供货
'''

text2 = '''
Vysoké Tatry sú najvyššie pohorie na Slovensku a v Poľsku a sú zároveň jediným 
horstvom v týchto štátoch s alpským charakterom. 
'''

translator = Translator()

dt1 = translator.detect(text1)
print(dt1)

dt2 = translator.detect(text2)
print(dt2)

In [ ]:
# Using rpy2
# Google API KEY:::
#api.key <- ''

# Read in 'database' tab from F21 Discounts
#df <- read_csv("Discounts_F21_data.csv",
#              locale = locals())

# Convert Headers:
#names(df) <- make.names(names(df))

# Translate
#test <- translate(content.vec = dfsTier.1.ws.Account.Name,
#                  google.apikey
#                  = my.api.key,
#                  sourrce.lang = 'en',
#                  target.lang ='de')

In [ ]:
# Upload NavInfo and F20 Discount Data.xlsx file locally
#from google.colab import files
#uploaded = files.upload()

###Optical Character Recognition (OCR)
B-F Distribution Agreement 
* Reading contents of pdf 
* Using OCR / Python libraries:
  * pytesseract, pdf2image


In [ ]:
try:
  from PIL import Image
except ImportError:
  import Image
import pytesseract

def ocr_core(filename):
  """
  This function handles core OCR image processing.
  """
  text = pytesseract.image_to_string(Image.open(filename))
  return text

print(ocr_core('/content/drive/MyDrive/China NavInfo Data/B-F Distribution Agreement.pdf'))

In [ ]:
# importing required modules
import PyPDF2
	
# creating a pdf file object
pdfFileObj = open('/content/drive/MyDrive/China NavInfo Data/Brown-Forman Scotch Products Distribution Agreement 百富门单一麦芽威士忌经销协议 [EXECUTION VERSION] Fully Signed NOA.pdf', 'rb')
	
# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
	
# printing number of pages in pdf file
print(pdfReader.numPages)
	
# creating a page object
pageObj = pdfReader.getPage(0)
	
# extracting text from page
print(pageObj.extractText())
	
# closing the pdf file object
pdfFileObj.close()


In [ ]:
#3 When finished
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')